# Project 3 - Sequence models

igu011 and edj001

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from datetime import datetime
from torch.utils.data import DataLoader, TensorDataset
from rich import print

import numpy as np
import torchtext
from os import listdir
import re
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

torch.manual_seed(265)
torch.set_default_dtype(torch.double)

device = torch.device("cpu")
print(f"Device {device}.")


Device cpu.

## 2.1 Word embedding

#### 1 - Read txt files and tokenize them to obtain train/validation/test lists of words.


In [2]:
tokenizer = get_tokenizer("basic_english")


def read_files(datapath="./"):
    """
    Return a list of strings, one for each line in each .txt files in 'datapath'
    """
    files = listdir(datapath)
    files = [datapath + f for f in files if f.endswith(".txt")]

    lines = []
    for f_name in files:
        with open(f_name) as f:
            lines += f.readlines()
    return lines


no_digits = "\w*[0-9]+\w*"
no_names = "\w*[A-Z]+\w*"
no_spaces = "\s+"


def tokenize(lines):
    """
    Tokenize the list of lines
    """
    list_text = []
    for line in lines:
        list_text += tokenizer(line)
    return list_text


def yield_tokens(lines):
    """
    Yield tokens, ignoring names and digits to build vocabulary
    """
    for line in lines:
        line = re.sub(no_digits + "|" + no_names, " ", line)
        line = re.sub(no_spaces, " ", line)
        yield tokenizer(line)


def count_freqs(data, vocab):
    """
    Count occurrences of each word in vocabulary in the data
    """
    freqs = torch.zeros(len(vocab), dtype=torch.int)
    for w in data:
        freqs[vocab[w]] += 1
    return freqs


train_books = read_files(datapath="./data_train/")
train_tokenized = tokenize(train_books)

val_books = read_files(datapath="./data_val/")
val_tokenized = tokenize(val_books)

test_books = read_files(datapath="./data_test/")
test_tokenized = tokenize(test_books)


#### 2 - Define a vocabulary based on the training dataset
To avoid getting a too large vocabulary, a solution can be to keep only words that appear at least 100 times in the training dataset.
Report the total number of words in the training dataset, the number of distinct words in the
training dataset and the size the defined vocabulary. Comment your results.

In [8]:
specials = ["<unk>", ",", ".", "!", "?"]
vocab = build_vocab_from_iterator(yield_tokens(train_books), min_freq=100, specials=specials)

vocab.append_token("i")

vocab.set_default_index(vocab["<unk>"])
vocab_size = len(vocab)

print("Total number of words in the dataset:   ", len(train_tokenized))
print("Number of distinct words in the dataset:", len(set(train_tokenized)))
print("Size the defined vocabular:             ", vocab_size)


freqs = count_freqs(train_tokenized, vocab)
top20 = freqs[len(specials) : len(specials) + 19]
print(
    "Top 20 occurences:\n",
    [
        (f.item(), w)
        for (f, w) in zip(top20, vocab.lookup_tokens(range(len(specials), len(specials) + 19)))
    ],
)


Total number of words in the dataset:    1368807

Number of distinct words in the dataset: 30374

Size the defined vocabular:              1050

Top 20 occurences:

[
    (71106, 'the'),
    (43426, 'and'),
    (33952, 'to'),
    (30061, 'of'),
    (23575, 'a'),
    (18657, 'in'),
    (20755, 'he'),
    (16814, 'that'),
    (15056, 'was'),
    (14400, 'his'),
    (13815, 'it'),
    (10997, 'with'),
    (10735, 'had'),
    (9430, 'her'),
    (9334, 'not'),
    (10562, 'you'),
    (9198, 'as'),
    (9152, 'at'),
    (8447, 'him')
]

In [17]:
def create_dataset(text, vocab, context_size=3):
    """
    Create a dataset from a list of context words and a list of target words
    """

    contexts = []
    targets = []
    n_text = len(text)

    txt = [vocab[w] for w in text]

    for i in range(n_text - context_size):

        t = txt[i + context_size]

        c = torch.Tensor(txt[i : i + context_size]).type(torch.long)

        targets.append(t)

        contexts.append(c)

    contexts = torch.stack(contexts)
    targets = torch.tensor(targets)
    return TensorDataset(contexts, targets)


context_size = 3

train_data = create_dataset(train_tokenized, vocab, context_size=context_size)
val_data = create_dataset(val_tokenized, vocab, context_size=context_size)
test_data = create_dataset(test_tokenized, vocab, context_size=context_size)

batch_size = 64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


#### 3 - Define a n-gram language model architecture based on this vocabulary that contains an embedding layer.
To drastically reduce computational cost, the dimension of the embedding can be as low as 16 even though in a real setting a larger space would be used.

In [18]:
class NGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((inputs.shape[0], -1))
        out = self.linear1(embeds)
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


#### 4 - Train several models, select the best one and evaluate its performance.
Note that the accuracy here is potentially extremely low, but the real objective is not to train a good predictor, only
to have a good representation of the semantic of each word in the vocabulary

In [19]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    """
    Train our model and save weight values
    """
    n_batch = len(train_loader)
    losses_train = []
    model.train()
    optimizer.zero_grad(set_to_none=True)

    for epoch in range(1, n_epochs + 1):

        loss_train = 0.0
        for contexts, labels in train_loader:

            contexts = contexts.to(device=device)
            labels = labels.to(device=device)

            outputs = model(contexts)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_train += loss.item()

        losses_train.append(loss_train / n_batch)

        print(
            "{}  |  Epoch {}  |  Training loss {:.5f}".format(
                datetime.now().time(), epoch, loss_train / n_batch
            )
        )
    return


def relative_error(a, b):
    return torch.norm(a - b) / torch.norm(a)


In [20]:
loss_fn = nn.NLLLoss()

n_epochs = 4
lr = 0.1
embedding_dim = 16

NGram_model = NGram(vocab_size, embedding_dim, context_size).to(device=device)
optimizer = optim.SGD(NGram_model.parameters(), lr=lr)

weight = train(
    n_epochs=n_epochs,
    optimizer=optimizer,
    model=NGram_model,
    loss_fn=loss_fn,
    train_loader=train_loader,
)


22:03:24.323818  |  Epoch 1  |  Training loss 4.17109

22:04:19.398517  |  Epoch 2  |  Training loss 3.96512

22:05:15.127855  |  Epoch 3  |  Training loss 3.89744

22:06:10.091023  |  Epoch 4  |  Training loss 3.85711

In [22]:
n_epochs = 15
lr = 0.1
embedding_dim = 64

NGram_larger_model = NGram(vocab_size, embedding_dim, context_size).to(device=device)
NGram_larger_optimizer = optim.SGD(NGram_larger_model.parameters(), lr=lr)

NGram_larger_model_weight = train(
    n_epochs=n_epochs,
    optimizer=NGram_larger_optimizer,
    model=NGram_larger_model,
    loss_fn=loss_fn,
    train_loader=train_loader,
)


22:10:17.976390  |  Epoch 1  |  Training loss 4.00733

22:11:14.258512  |  Epoch 2  |  Training loss 3.83581

22:12:05.433564  |  Epoch 3  |  Training loss 3.78053

22:12:54.034753  |  Epoch 4  |  Training loss 3.74690

22:13:54.970527  |  Epoch 5  |  Training loss 3.72303

...

22:21:33.158373  |  Epoch 11  |  Training loss 3.64975

22:22:59.468252  |  Epoch 12  |  Training loss 3.64257

22:24:02.836666  |  Epoch 13  |  Training loss 3.63613

22:25:05.612254  |  Epoch 14  |  Training loss 3.63028

22:26:08.133481  |  Epoch 15  |  Training loss 3.62495

In [23]:
def accuracy(model, loader):
    """
    Compute the accuracy of model with given data loader
    """
    model.eval()
    correct = 0
    with torch.no_grad():
        for contexts, labels in loader:
            contexts = contexts.to(device=device)
            labels = labels.to(device=device)

            outputs = model(contexts)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    return correct / len(loader.dataset)

In [24]:
def model_selection(models, titles, train_loader, val_loader):
    """
    Choose the best model from the list of models based on validation accuracy
    """
    best_model = None
    best_acc = 0
    for model, title in zip(models, titles):
        acc = accuracy(model, val_loader)
        train_acc = accuracy(model, train_loader)
        print(f"{title} | Train accuracy {train_acc:.2%} |  Validation accuracy {acc:.2%}")
        if acc > best_acc:
            best_model = model
            best_acc = acc
    return best_model


In [25]:
best_model = model_selection(
    [NGram_model, NGram_larger_model], ["NGram", "NGram_larger"], train_loader, val_loader
)


best_model_acc = accuracy(best_model, val_loader)


NGram | Train accuracy 20.35% |  Validation accuracy 21.59%

NGram_larger | Train accuracy 21.80% |  Validation accuracy 22.79%

#### 5 - Compute the cosine similarity matrix of the vocabulary based on the trained embedding.
For some words of your choice (e.g. me, white, man, have, be, child, yes, what etc.), report the 10
most similar words. Comment your results.

In [26]:
best_model_weights = best_model.embeddings.weight.detach()

sim_matrix = np.zeros(shape=(len(best_model_weights), len(best_model_weights)))

for w1 in range(0, len(best_model_weights)):
    for w2 in range(0, len(best_model_weights)):
        # Cosine similarity
        sim_matrix[w1][w2] = np.dot(best_model_weights[w1], best_model_weights[w2]) / (
            np.linalg.norm(best_model_weights[w1]) * np.linalg.norm(best_model_weights[w2])
        )


In [29]:
testing_words = ["me", "white", "man", "have", "be", "child", "yes", "what"]
testing_words_idx = [(i, vocab.get_stoi()[i]) for i in testing_words]

n_most_similar = 10
for w in testing_words_idx:

    indices = (-sim_matrix[w[1]]).argsort()[:n_most_similar]
    sim_words = [(vocab.get_itos()[i], sim_matrix[w[1]][i]) for i in indices]
    print(f"{w} most similar words:")

    for sim_w in sim_words:
        print(f"\t{sim_w}")


('me', 36) most similar words:

('me', 1.0000000000000002)

('wish', 0.4193623266798826)

('him', 0.40000229386481717)

('start', 0.39847470893917614)

...

('act', 0.3224882860219756)

('guns', 0.3124082989122353)

('position', 0.3097287048350473)

('whether', 0.29957600440574095)

('such', 0.2964352709393147)

TODO: COMMENT HERE

#### 6 -Visualize the embedding space.
To do so, upload the vocabulary and their corresponding values in the embedding space as tsv files. Try to find
and select clusters. Report both plots and their corresponding selections for some meaningful
clusters.

In [30]:
import csv

embedding_weights = NGram_larger_model.embeddings.weight.detach()

with open("./weights.tsv", "wt") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i in embedding_weights:
        tsv_writer.writerow(i.numpy())


with open("./metadata.tsv", "wt") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    vocab_dict = vocab.get_itos()
    for i in vocab_dict:
        tsv_writer.writerow([i])


TODO: PUT VISUALIZATIONS HERE

TODO: COMMENT IN REGARDS TO 5

## 2.2 Conjugating *be* and *have*

#### 1 - Use your trained word embedding and define a MLP architecture as well as a RNN architecture to predict be and have conjugation given the context around the missing target. 
Use the same context size for both MLPs and RNNs, even though RNNs could take a context size of arbitrary
length.

#### 2 - Train several models, select the best one and evaluate its performance. 
Comment the differences in terms of performances/training time between the MLP architecture and the RNN
architecture.

## 2.3 Text generation

#### 1 -  Use your trained word embedding and define a RNN architecture that can predict the next word given the context before the target.


#### 2 - Train several models, select the best one and evaluate its performance.
Note that the accuracy here is potentially very low.


#### 3 - Implement the beam search algorithm.

#### 4 - Have fun playing with your model. 
Start a sentence, give it to your model, and let it complete the next n words using the beam search algorithm. Report and comment your results.